In [40]:
import numpy as np
import pandas as pd
df1 = pd.read_excel('../data/Sex & Age by Year.xlsx',sheet_name='Person',header=2)
display(df1)
area_df = df1[["SA2 code","SA2 name","Postcode"]]
area_df.info()

,Year,S/T code,S/T name,GCCSA code,GCCSA name,SA4 code,SA4 name,SA3 code,SA3 name,SA2 code,...,no..9,no..10,no..11,no..12,no..13,no..14,no..15,no..16,no..17,no..18
0,2001,2,Victoria,2RVIC,Rest of Vic.,201,Ballarat,20101,Ballarat,21001,...,435,368,247,233,184,168,151,67,42,5756
1,2001,2,Victoria,2RVIC,Rest of Vic.,201,Ballarat,20101,Ballarat,21002,...,783,743,513,448,371,440,403,323,373,11497
2,2001,2,Victoria,2RVIC,Rest of Vic.,201,Ballarat,20101,Ballarat,21003,...,1271,1238,913,798,729,723,615,442,387,20268
3,2001,2,Victoria,2RVIC,Rest of Vic.,201,Ballarat,20101,Ballarat,21004,...,1272,1144,872,785,800,820,689,429,310,20424
4,2001,2,Victoria,2RVIC,Rest of Vic.,201,Ballarat,20101,Ballarat,21005,...,407,379,248,179,142,124,84,39,24,5320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9697,2021,2,Victoria,2RVIC,Rest of Vic.,217,Warrnambool and South West,21703,Colac - Corangamite,21476,...,264,303,363,422,404,311,182,103,71,3978
9698,2021,2,Victoria,2RVIC,Rest of Vic.,217,Warrnambool and South West,21704,Warrnambool,21477,...,444,477,555,526,475,339,210,153,113,6989
9699,2021,2,Victoria,2RVIC,Rest of Vic.,217,Warrnambool and South West,21704,Warrnambool,21478,...,618,675,735,731,781,671,420,228,252,9963
9700,2021,2,Victoria,2RVIC,Rest of Vic.,217,Warrnambool and South West,21704,Warrnambool,21479,...,1374,1435,1444,1404,1245,1150,843,543,504,22462


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9702 entries, 0 to 9701
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SA2 code  9702 non-null   int64  
 1   SA2 name  9702 non-null   object 
 2   Postcode  8400 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 227.5+ KB


In [39]:
df2 = pd.DataFrame( {"year":df1["Year"], "area_code":df1["SA2 code"], "Total": df1[ df1.columns[-1]  ]} )  #df1[["Year","SA2 code",]]
df2

,year,area_code,Total
0,2001,21001,5756
1,2001,21002,11497
2,2001,21003,20268
3,2001,21004,20424
4,2001,21005,5320
...,...,...,...
9697,2021,21476,3978
9698,2021,21477,6989
9699,2021,21478,9963
9700,2021,21479,22462


In [33]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import  PolynomialFeatures
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
def predict_by_model(X,Y,last_population):
    lr  = linear_model.BayesianRidge()
    lr.fit(X, Y)
    y_predict = lr.predict(X)
    tmp_x = list(X[-1][1:]) + list([Y[-1]])
    predict_arr = []
    years = list(range(2022,2030))
    population_arr = []
    for year in years:
        predict_res = lr.predict([tmp_x])[0]
        predict_arr.append(predict_res)
        tmp_x.pop(0)
        tmp_x.append(predict_res)
        last_population = last_population*(1+predict_res)
        population_arr.append(last_population)
    return dict(zip(years,population_arr))

In [34]:
def split_xy_from_pct_change_arr(pct_change_arr):
    x_num = 9
    x_arr = []
    y_arr = []
    arr = list(pct_change_arr)
    for i in range(x_num+1+1,len(arr)):
        x  = arr[i-(x_num+1):i-1]
        y = arr[i-1]
        x_arr.append(x)
        y_arr.append(y)
    X = np.array(x_arr)
    Y = y_arr
    return X,Y
def predict_population(df):
    try:
        pct_change_arr = pd.Series.pct_change(df['Total']).values[1:]
        X,Y = split_xy_from_pct_change_arr(pct_change_arr)
        last_population = df['Total'].values[-1]
        dict_res = predict_by_model(X,Y,last_population)
    except:
        return None
    old = dict(zip(df["year"].values,df["Total"].values))
    dict_res.update( old   )
    return pd.Series(dict_res)

In [ ]:
s_dict = dict()
for area_code,sub_df in df2.groupby("area_code"):
    print("area_code:",area_code)
    s = predict_population(sub_df)
    if s is not None:
        s_dict[area_code] = s 

In [36]:
pd.DataFrame(s_dict).sort_index().T

,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,...,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029
21001,5756.0,6092.0,6293.0,6480.0,6648.0,6761.0,7034.0,7272.0,7614.0,7894.0,...,15472.0,16823.0,17992.558565,19243.327512,20580.879902,22011.569396,23541.885011,25178.272842,26928.540748,28800.402168
21002,11497.0,11708.0,12015.0,12189.0,12269.0,12356.0,12408.0,12480.0,12476.0,12462.0,...,12183.0,12076.0,12056.200144,12036.434411,12016.538832,11996.736408,11977.043851,11957.255415,11937.502182,11917.988228
21003,20268.0,20586.0,20838.0,21000.0,21134.0,21279.0,21492.0,21736.0,21937.0,22215.0,...,24299.0,24348.0,24565.153010,24789.697468,25015.557664,25244.095773,25472.207322,25698.807802,25929.730585,26161.612007
21004,20424.0,20389.0,20407.0,20314.0,20423.0,20534.0,20911.0,21470.0,21940.0,22666.0,...,25395.0,25269.0,25503.760359,25741.908730,25977.263431,26213.895408,26442.257068,26666.646538,26896.002844,27139.462694
21005,5320.0,5399.0,5557.0,5620.0,5857.0,6037.0,6131.0,6252.0,6431.0,6595.0,...,7369.0,7232.0,7174.252097,7043.177611,6878.920777,6623.196260,6310.686634,5900.782192,5442.778764,4895.382407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21476,3452.0,3479.0,3511.0,3511.0,3492.0,3459.0,3489.0,3501.0,3490.0,3460.0,...,3905.0,3978.0,4089.294895,4243.264158,4421.760148,4639.734333,4896.240764,5177.563075,5526.805808,5941.090862
21477,6718.0,6704.0,6676.0,6643.0,6638.0,6652.0,6606.0,6631.0,6703.0,6739.0,...,6873.0,6989.0,7030.701192,7049.848028,7038.803926,7009.663180,6982.648146,6959.950077,6960.102018,6987.196474
21478,8317.0,8387.0,8450.0,8487.0,8517.0,8601.0,8694.0,8792.0,8878.0,8965.0,...,9842.0,9963.0,9984.444279,10051.206187,10050.526443,10068.844336,10146.758658,10249.557107,10398.746465,10639.676326
21479,17053.0,17449.0,17726.0,17937.0,18172.0,18528.0,18877.0,19107.0,19369.0,19634.0,...,22379.0,22462.0,22657.988663,22821.575524,22992.548244,23168.774759,23346.118191,23526.307878,23692.622320,23847.321825
